In [ ]:
# # save_results.py
# import json
# from pathlib import Path

# # Create the results dictionary with values from the log
# results = {
#     'cv_results': [
#         {
#             'fold': 0,
#             'best_val_loss': 0.0403,
#             'final_val_acc': 98.84
#         },
#         {
#             'fold': 1,
#             'best_val_loss': 0.0426,
#             'final_val_acc': 98.69
#         },
#         {
#             'fold': 2,  # Early stopped
#             'best_val_loss': 4.8652,
#             'final_val_acc': 38.04
#         },
#         {
#             'fold': 3,  # Early stopped
#             'best_val_loss': 4.7516,
#             'final_val_acc': 36.40
#         },
#         {
#             'fold': 4,
#             'best_val_loss': 0.0405,
#             'final_val_acc': 98.80
#         }
#     ],
#     'num_classes': 67,
#     'window_length': 500,
#     'best_fold': 0,  # Fold 1 had the best performance
#     'best_val_loss': 0.0403,  # From fold 1
#     'best_val_acc': 98.84  # From fold 1
# }

# # Create logs directory if it doesn't exist
# log_dir = Path("logs")
# log_dir.mkdir(exist_ok=True)

# # Save results
# with open(log_dir / "training_results.json", 'w') as f:
#     json.dump(results, f, indent=4)

# print("Training results saved successfully!")

Training results saved successfully!


In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
import torch

from utils.config import Config
from model import IMUCNN, TimeSeriesAugmentation
from utils.data_processing import SignalPreprocessor, ExerciseDataset
from utils.training import train_model_with_cv, optimize_for_mobile

# Load configuration
config = Config()

# Load data
print("Loading data...")
data = pd.read_pickle(config.data_dir / 'model_data_window_500_space_50.pkl')

# Set number of classes based on data
config.num_classes = len(data['activity_name'].unique())
print(f"Number of classes: {config.num_classes}")

# Add sampling rate information to dataset
data['sampling_rate'] = data['dataset'].map({
    'mmfit': 100,
    'har_data': 100,
    'reco': 50
})

# Train with cross-validation
# print("Starting cross-validation training...")
# cv_results = train_model_with_cv(IMUCNN, data, config)

Using device: mps
Loading data...
Number of classes: 67


In [2]:
data

,id,sig_array,activity_name,subject_id,dataset,sampling_rate
0,0,"[[-0.1832595, -0.2076941, -0.53145254, 9.77168...",squats,0,mmfit,100
1,1,"[[1.502728, -1.7263045, -1.4062113, 0.86939216...",squats,0,mmfit,100
2,2,"[[-0.21502449, -1.5980228, -1.4941758, 6.47373...",non-e,0,mmfit,100
3,3,"[[0.6059781, 0.07819072, -0.53633946, 7.046148...",non-e,0,mmfit,100
4,4,"[[0.09896013, -0.2724458, 0.53145254, 1.087338...",non-e,0,mmfit,100
...,...,...,...,...,...,...
151782,151782,"[[0.6453633617195623, -3.3918659447031194, 0.6...",deviceontable,reco_10006,reco,50
151783,151783,"[[0.5943834536410079, -3.3774912748098864, 0.6...",deviceontable,reco_10006,reco,50
151784,151784,"[[0.5968571751596301, -3.3746797423858386, 0.6...",deviceontable,reco_10006,reco,50
151785,151785,"[[0.6648895340216258, -3.32562119567251, 0.689...",deviceontable,reco_10006,reco,50


In [ ]:
# # optimize_mobile.py
# import torch
# import json
# from pathlib import Path
# from model import IMUCNN

# def optimize_for_mobile(model, config):
#     """Optimize model for mobile deployment using TorchScript only"""
#     print("Optimizing model for mobile deployment...")
    
#     # Move model to CPU and eval mode
#     model = model.cpu()
#     model.eval()
    
#     # Create example input
#     example_input = torch.randn(
#         1, 6, config['window_length'],
#         device='cpu'
#     )
    
#     # Export to TorchScript
#     print("Tracing model...")
    
#     with torch.no_grad():
#         # Trace the model
#         traced_model = torch.jit.trace(model, example_input)
        
#         # Test the traced model
#         print("Validating traced model...")
#         original_output = model(example_input)
#         traced_output = traced_model(example_input)
        
#         if torch.allclose(original_output, traced_output):
#             print("Traced model validated successfully!")
#         else:
#             print("Warning: Traced model outputs differ from original model!")
    
#     # Save optimized model
#     output_path = Path("models") / "mobile_optimized_model.pt"
#     traced_model.save(str(output_path))
#     print(f"Saved optimized model to {output_path}")
    
#     # Calculate and log model sizes
#     model_path = Path("models") / f"best_model_fold_{config['best_fold']}.pt"
#     original_size = model_path.stat().st_size
#     optimized_size = output_path.stat().st_size
    
#     print(f"\nModel size comparison:")
#     print(f"Original model size: {original_size/1024:.2f} KB")
#     print(f"Optimized model size: {optimized_size/1024:.2f} KB")
#     print(f"Size reduction: {100*(1-optimized_size/original_size):.1f}%")
    
#     return traced_model


In [ ]:
# # Load training results
# print("Loading training results...")
# with open("logs/training_results.json", 'r') as f:
#     results = json.load(f)

# print(f"Loaded results - Best fold: {results['best_fold']}")
# print(f"Best validation accuracy: {results['best_val_acc']}%")

# # Initialize model
# model = IMUCNN(
#     num_classes=results['num_classes'],
#     window_length=results['window_length']
# )

# # Load best model weights
# model_path = Path("models") / f"best_model_fold_{results['best_fold']}.pt"
# print(f"Loading model from {model_path}")
# model.load_state_dict(torch.load(model_path, map_location='cpu', weights_only=True))

# # Optimize for mobile
# try:
#     mobile_model = optimize_for_mobile(model, results)
#     print("\nMobile optimization completed successfully!")
    
#     # Test the optimized model
#     print("\nTesting optimized model...")
#     test_input = torch.randn(1, 6, results['window_length'])
#     with torch.no_grad():
#         output = mobile_model(test_input)
#         print(f"Test output shape: {output.shape}")
#         print("Model successfully processes test input!")
        
# except Exception as e:
#     print(f"Error during mobile optimization: {str(e)}")
#     print("\nDetailed error information:")
#     import traceback
#     traceback.print_exc()

Loading training results...
Loaded results - Best fold: 0
Best validation accuracy: 98.84%
Loading model from models/best_model_fold_0.pt
Optimizing model for mobile deployment...
Tracing model...
Validating traced model...
Traced model validated successfully!
Saved optimized model to models/mobile_optimized_model.pt

Model size comparison:
Original model size: 16548.56 KB
Optimized model size: 16587.98 KB
Size reduction: -0.2%

Mobile optimization completed successfully!

Testing optimized model...
Test output shape: torch.Size([1, 67])
Model successfully processes test input!


In [1]:
# calculate_f1.py
import torch
import json
from pathlib import Path
from model import IMUCNN
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import KFold
import numpy as np
from tqdm import tqdm
import pandas as pd
from utils.data_processing import MemoryEfficientDataset, SignalPreprocessor

def evaluate_model(model, dataloader, device):
    """Evaluate model and get predictions"""
    model.eval()
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predictions = torch.max(outputs, 1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return np.array(all_predictions), np.array(all_labels)

In [3]:
# Load training results
print("Loading training results...")
with open("logs/training_results.json", 'r') as f:
    results = json.load(f)

# Load your data
print("Loading dataset...")
data = pd.read_pickle('data/model_data_window_500_space_50.pkl')

# Create indices for cross-validation (exactly as in training)
indices = np.arange(len(data))
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Get the validation indices for the best fold
best_fold = results['best_fold']
for fold, (train_idx, val_idx) in enumerate(kfold.split(indices)):
    if fold == best_fold:
        break

print(f"Creating validation dataset for fold {best_fold}")

# Create validation dataset
val_dataset = MemoryEfficientDataset(
    data,
    val_idx,
    preprocessor=SignalPreprocessor(target_sampling_rate=50),
    target_length=results['window_length']
)

# Create validation dataloader
val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# Initialize and load model
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

model = IMUCNN(
    num_classes=results['num_classes'],
    window_length=results['window_length']
).to(device)

model_path = Path("models") / f"best_model_fold_{best_fold}.pt"
print(f"Loading model from {model_path}")
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# Get predictions
print("\nGenerating predictions...")
predictions, labels = evaluate_model(model, val_loader, device)

# Calculate metrics
f1_macro = f1_score(labels, predictions, average='macro')
f1_weighted = f1_score(labels, predictions, average='weighted')

print("\nF1 Scores:")
print(f"Macro F1: {f1_macro:.4f}")
print(f"Weighted F1: {f1_weighted:.4f}")

# Get class names from dataset
class_names = val_dataset.label_encoder.classes_

# Print detailed classification report
print("\nDetailed Classification Report:")
print(classification_report(labels, predictions, target_names=class_names))

# Save results
metrics = {
    'f1_macro': float(f1_macro),
    'f1_weighted': float(f1_weighted),
    'classification_report': classification_report(labels, predictions, 
                                                target_names=class_names,
                                                output_dict=True)
}

metrics_path = Path("logs") / f"metrics_fold_{best_fold}.json"
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=4)

print(f"\nMetrics saved to {metrics_path}")

Loading training results...
Loading dataset...
Creating validation dataset for fold 0
Using device: mps


/var/folders/w3/3m5sc_792q1ccckrn_rdyfmh0000gn/T/ipykernel_98135/1897673065.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path,

Loading model from models/best_model_fold_0.pt

Generating predictions...


Evaluating: 100%|██████████| 949/949 [00:42<00:00, 22.38it/s] 


F1 Scores:
Macro F1: 0.9283
Weighted F1: 0.9874

Detailed Classification Report:
                                              precision    recall  f1-score   support

                     alternatingdumbbellcurl       0.00      0.00      0.00         3
                           armbandadjustment       0.96      0.72      0.82        60
                            bandpull-downrow       0.96      1.00      0.98        79
                                 bicep_curls       0.97      1.00      0.99        39
                                  bicepcurls       1.00      1.00      1.00       227
                            bicepscurl(band)       1.00      0.99      0.99        86
                                    boxjumps       1.00      1.00      1.00       619
                                      burpee       1.00      1.00      1.00       309
                                     burpees       1.00      1.00      1.00       677
                             butterflysit-up       1.00  


/Users/jackkrolik/.pyenv/versions/ds4440/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jackkrolik/.pyenv/versions/ds4440/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jackkrolik/.pyenv/versions/ds4440/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 